In [45]:
# Cell 0 – Fix scipy (run ONCE)
!pip install scipy --upgrade --quiet
import scipy
print("SciPy version:", scipy.__version__)


SciPy version: 1.16.3



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2  # Smaller than EfficientNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("TensorFlow:", tf.__version__)


TensorFlow: 2.20.0


In [47]:
IMG_SIZE = 180

datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)
datagen_test = ImageDataGenerator(rescale=1./255)

train_set = datagen_train.flow_from_directory(
    "../dataset/pokemon/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=32
)

test_set = datagen_test.flow_from_directory(
    "../dataset/pokemon/validation",
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=32
)

print("Train samples:", train_set.samples)
print("Val samples:", test_set.samples)
print("Classes:", len(train_set.class_indices))


Found 4978 images belonging to 150 classes.
Found 1842 images belonging to 150 classes.
Train samples: 4978
Val samples: 1842
Classes: 150


In [48]:
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(len(train_set.class_indices), activation='softmax')(x)

model = Model(inputs, outputs)
model.summary()


C:\Users\91916\AppData\Local\Temp\ipykernel_43000\2035857503.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)           │ (None, 180, 180, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 6, 6, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 150)                 │          19,350 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,441,302 (9.31 MB)

 Trainable params: 183,318 (716.09 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [49]:
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=4, min_lr=1e-7, verbose=1)
]


In [50]:
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_set,
    validation_data=test_set,
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 124s 758ms/step - accuracy: 0.0277 - loss: 4.9175 - val_accuracy: 0.0852 - val_loss: 4.5422 - learning_rate: 0.0010
Epoch 2/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 115s 738ms/step - accuracy: 0.1217 - loss: 4.1493 - val_accuracy: 0.2725 - val_loss: 3.3982 - learning_rate: 0.0010
Epoch 3/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 95s 609ms/step - accuracy: 0.2360 - loss: 3.3217 - val_accuracy: 0.4197 - val_loss: 2.6155 - learning_rate: 0.0010
Epoch 4/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 107s 680ms/step - accuracy: 0.3190 - loss: 2.8003 - val_accuracy: 0.5119 - val_loss: 2.1612 - learning_rate: 0.0010
Epoch 5/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 90s 576ms/step - accuracy: 0.3891 - loss: 2.4848 - val_accuracy: 0.5570 - val_loss: 1.8923 - learning_rate: 0.0010
Epoch 6/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 106s 676ms/step - accuracy: 0.4253 - loss: 2.2738 - val_accuracy: 0.5972 - val_loss: 1.7268 - learning_rate: 0.0010
Epoch 7/20
156/156 ━━━━━━━━━━━━━━━━━━━━ 113s 724ms/step - accuracy

In [51]:
# Best validation accuracy during training
best_val_acc = max(history.history['val_accuracy'])
print("Best val_accuracy during training:", best_val_acc)

# Final evaluation on validation set
val_loss, val_acc = model.evaluate(test_set)
print("Final validation loss:", val_loss)
print("Final validation accuracy:", val_acc)

# Also print training accuracy for comparison
best_train_acc = max(history.history['accuracy'])
print("Best training accuracy:", best_train_acc)


Best val_accuracy during training: 0.6900108456611633
58/58 ━━━━━━━━━━━━━━━━━━━━ 19s 329ms/step - accuracy: 0.6900 - loss: 1.2978
Final validation loss: 1.2977551221847534
Final validation accuracy: 0.6900108456611633
Best training accuracy: 0.6106870174407959


In [52]:
model.save("pokemon_mobilenetv2.keras")